In [2]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pymongo

In [4]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [5]:
# Define database and collection
db = client.mars_db
collection = db.articles

In [24]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(response.text, 'html.parser')

In [27]:
# Retrieve the parent divs for all articles
results = soup.find_all('div', class_='slide')

In [28]:
results

[<div class="slide">
 <div class="image_and_description_container">
 <a href="/news/8896/nasa-ingenuity-mars-helicopter-prepares-for-first-flight/">
 <div class="rollover_description">
 <div class="rollover_description_inner">
 Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps.  
 </div>
 <div class="overlay_arrow">
 <img alt="More" src="/assets/overlay-arrow.png"/>
 </div>
 </div>
 <img alt="NASA Ingenuity Mars Helicopter Prepares for First Flight" class="img-lazy" data-lazy="/system/news_items/list_view_images/8896_thumb-320.jpg" src="/assets/loading_320x240.png"/>
 </a>
 </div>
 <div class="content_title">
 <a href="/news/8896/nasa-ingenuity-mars-helicopter-prepares-for-first-flight/">
 NASA Ingenuity Mars Helicopter Prepares for First Flight
 </a>
 </div>
 </div>,
 <div class="slide">
 <div class="image_and_description_container">
 <a href="/news/8882/nasas-perseverance-drives-on-mars-terrain-for-first-time/">
 <div class="ro